## 1. Dataset
Got Dataset from _[Kaggle](https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset/discussion)_ check [images] folder for train/validation data sets

## 2. Install libraries, packages and dataset
import matplotlib for plotting and displaying images and import torch for using Pytorch for CNN

In [ ]:
!pip install timm
!pip install --upgrade opencv-contrib-python
!pip install -U git+https://github.com/albumentations-team/albumentations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import random

from torchvision.datasets import ImageFolder
from torchvision import transforms as t

## 3. Configurations
Complete data loading as well as data augumentation. Transform train set images to get variety to enrich our training process with Compose, Flip, and Rotate. Also, converts image format to Tensor format with toTensor() for both train set and validation set.

In [ ]:
TRAIN_IMG_FOLDER_PATH = 'images/train'
VALID_IMG_FOLDER_PATH = 'images/validation'

In [ ]:
train_aug = t.Compose([
    t.RandomHorizontalFlip(p=0.5),
    t.RandomRotation(degrees=(-20, +20)),
    t.ToTensor()
])


valid_aug = t.Compose([
    t.ToTensor()
])

In [ ]:
train_set = ImageFolder(TRAIN_IMG_FOLDER_PATH, transform = train_aug)
valid_set = ImageFolder(VALID_IMG_FOLDER_PATH, transform = valid_aug)

print(f"Total no. of examples in train set : {len(train_set)}")
print(f"Total no. of examples in valid set : {len(valid_set)}")

print(train_set.class_to_idx)

# Printing random example to checking step
def print_random_img(train_set):
    random.seed(299)
    rand_idx = random.random()
    image, label = train_set[int(rand_idx * len(train_set))]
    plt.imshow(image.permute(1, 2, 0))
    for idx, emotion in enumerate(train_set.class_to_idx.keys()):
      if idx == label:
        label = emotion
    plt.title(label)
    plt.show()

print_random_img(train_set=train_set)

In [ ]:
# Load datasets
LR = 0.001  # Learning rate to be used during training
BATCH_SIZE = 32  # The number of images to take during each training iteration
EPOCHS = 50  # Determines the amount of training
DEVICE='cpu'  # Enables you to perform compute-intensive operations faster by parallelizing tasks across GPUs
MODEL_NAME='efficientnet_b0'  # A CNN model that is pre-trained on more than a million images from the ImageNet database

# We will use DataLoader to load 32 images at a time to our CNN training model
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size  = BATCH_SIZE)

# Analyzing one batch
for images, labels in train_loader:
    break

print(f"One image batch shape : {images.shape}")
print(f"One label batch shape : {labels.shape}")

## 4. Creating Models

In [ ]:
import timm
from torch import nn

class FaceModel(nn.Module):
  def __init__(self):
    super(FaceModel, self).__init__()

    # Creating a NN model
    self.eff_net = timm.create_model('efficientnet_b0', pretrained = True, num_classes = 7)

  def forward(self, images, labels = None):
    logits = self.eff_net(images)

    if labels is not None:
      loss = nn.CrossEntropyLoss()(logits, labels)  # Calculates the loss
      return logits, loss

    return logits

In [ ]:
model = FaceModel()
model.to(DEVICE)

## 5. Create Train and Eval Function
Optimizers are used to improve the model's accuracy with each iteration. Logits are the raw predictions (outputs) of our model. Loss quantifies how bad the model predicts (how far from the actual). With this information, we improve the model's accuracy. Also, we still get the logits and loss to calculate the model's performance, but we are not attempting to improve the model's accuracy here.

In [ ]:
from tqdm import tqdm

In [ ]:
def multiclass_accuracy(y_pred, y_true):
  """Creating a tensor (like an array) that compares predicted value with
  actual value (assigning 1 if accurate, 0 if not),
  then returning the mean value as accuracy
  """
  top_p, top_class = y_pred.topk(1,dim=1)
  equals = top_class == y_true.view(*top_class.shape)
  return torch.mean(equals.type(torch.FloatTensor))

def train_fn(model, dataloader, optimizer, current_epo):
  """Training function that trains the model, with the given optimizer, using
  the given dataloader. During the training, it also calculates model's
  total loss and total accuracy on training set, and returns these values.
  """
  model.train()
  total_loss = 0.0
  total_acc = 0.0
  tk = tqdm(dataloader, desc = 'EPOCH' + "[TRAIN]" + str(current_epo + 1) + "/" +str(EPOCHS))

  for t, data in enumerate(tk):
    images, labels = data
    images, labels = images.to(DEVICE), labels.to(DEVICE)

    optimizer.zero_grad()

    logits, loss = model(images, labels)

    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_acc += multiclass_accuracy(logits, labels)
    tk.set_postfix({'loss': '%6f' %float(total_loss/(t+1)), 'acc': '%6f' %float(total_acc / (t+1))})

  return total_loss / len(dataloader), total_acc / len(dataloader)

In [ ]:
def eval_fn(model, dataloader, current_epo):
  """Evaluation function that validates the model, with the given optimizer, using
  the given dataloader. During the evaluation, it also calculates model's
  total loss and total accuracy on validation set, and returns these values.
  """
  model.eval()
  total_loss = 0.0
  total_acc = 0.0
  tk = tqdm(dataloader, desc = 'EPOCH' + "[VALID]" + str(current_epo + 1) + "/" +str(EPOCHS))

  for t, data in enumerate(tk):
    images, labels = data
    images, labels = images.to(DEVICE), labels.to(DEVICE)

    logits, loss = model(images, labels)

    total_loss += loss.item()
    total_acc += multiclass_accuracy(logits, labels)
    tk.set_postfix({'loss': '%6f' %float(total_loss/(t+1)), 'acc': '%6f' %float(total_acc / (t+1))})

  return total_loss / len(dataloader), total_acc / len(dataloader)

In this section, we implement the Adam algorithm and Stochastic Optimization to create the training loop. We also keep track of the best validation loss in order to continually update it. The number of epochs is set to 50, so the training loop iterates for a total of 50 times. With each iteration, we expect both the training loss and validation loss to decrease gradually as our model becomes more accurate.

We specifically focus on minimizing the validation loss rather than the training loss because our ultimate goal is to have a model that performs well in real-life scenarios beyond just our training data. While accuracy can be subjectively biased, measuring how close we are to reality through minimizing loss provides us with a better understanding of truthfulness.

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = LR)

best_valid_loss = np.Inf

for i in range (EPOCHS):
  train_loss, train_acc = train_fn(model, train_loader, optimizer, i)
  valid_loss, valid_acc = eval_fn(model, valid_loader, i)

  if valid_loss < best_valid_loss:
    torch.save(model.state_dict(), "best.weights.pt")
    print("SAVED-BEST-WEIGHTS")
    best_valid_loss = valid_loss

## 6. Data Visualization
Drawing plots using Matplotlib about testing, training accuracy and testing, training loss for each Epoch

In [ ]:
plt.plot(EPOCHS,train_acc, label = "Training Accuracy")
plt.plot(EPOCHS,valid_acc, label = "Testing Accuracy")
plt.title("Testing Accuracy & Training Accuracy for each epoch")
plt.ylabel("Accuracy (%)")
plt.xlabel("Epochs")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
plt.plot(EPOCHS, train_loss, label = "Training Loss")
plt.plot(EPOCHS, valid_loss, label = "Test Loss")
plt.title("Testing Loss & Training Loss for each epoch")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
def view_classify(img, ps):

    classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    ps = ps.data.cpu().numpy().squeeze()
    img = img.numpy().transpose(1,2,0)

    fig, (ax1, ax2) = plt.subplots(figsize=(5,9), ncols=2)
    ax1.imshow(img)
    ax1.axis('off')
    ax2.barh(classes, ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(classes)
    ax2.set_yticklabels(classes)
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)